# Context Enrichment Window for Document Retrieval

## Overview

This code implements a context enrichment window technique for document retrieval in a vector database. It enhances the standard retrieval process by adding surrounding context to each retrieved chunk, improving the coherence and completeness of the returned information.

## Motivation

Traditional vector search often returns isolated chunks of text, which may lack necessary context for full understanding. This approach aims to provide a more comprehensive view of the retrieved information by including neighboring text chunks.

## Key Components

1. PDF processing and text chunking
2. Vector store creation using FAISS and OpenAI embeddings
3. Custom retrieval function with context window
4. Comparison between standard and context-enriched retrieval

## Method Details

### Document Preprocessing

1. The PDF is read and converted to a string.
2. The text is split into chunks with surrounding sentences

### Vector Store Creation

1. OpenAI embeddings are used to create vector representations of the chunks.
2. A FAISS vector store is created from these embeddings.

### Context-Enriched Retrieval

LlamaIndex has a special parser for such task. [SentenceWindowNodeParser](https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/#sentencewindownodeparser) this parser splits documents into sentences. But the resulting nodes inculde the surronding senteces with a relation structure. Then, on the query [MetadataReplacementPostProcessor](https://docs.llamaindex.ai/en/stable/module_guides/querying/node_postprocessors/node_postprocessors/#metadatareplacementpostprocessor) helps connecting back these related sentences.

### Retrieval Comparison

The notebook includes a section to compare standard retrieval with the context-enriched approach.

## Benefits of this Approach

1. Provides more coherent and contextually rich results
2. Maintains the advantages of vector search while mitigating its tendency to return isolated text fragments
3. Allows for flexible adjustment of the context window size

## Conclusion

This context enrichment window technique offers a promising way to improve the quality of retrieved information in vector-based document search systems. By providing surrounding context, it helps maintain the coherence and completeness of the retrieved information, potentially leading to better understanding and more accurate responses in downstream tasks such as question answering.

<div style="text-align: center;">

<img src="../images/vector-search-comparison_context_enrichment.svg" alt="context enrichment window" style="width:70%; height:auto;">
</div>

### Import libraries and environment variables

In [4]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceWindowNodeParser, SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
import faiss
import os
import sys
from dotenv import load_dotenv
from pprint import pprint

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
#os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')



#ChatGroq(model="llama-3.1-8b-instant", temperature=0)

#CohereEmbeddings(model="embed-english-v3.0")
# Llamaindex global settings for llm and embeddings
EMBED_DIMENSION=512


from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings
Settings.llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
Settings.embed_model = CohereEmbeddings(model="embed-english-v3.0")

### Read docs

In [5]:
path = "../data/"
reader = SimpleDirectoryReader(input_dir=path, required_exts=['.pdf'])
documents = reader.load_data()
print(documents[0])

Doc ID: 1eca7ccb-57a3-406e-86f2-5aaeb980c1f0
Text: Understanding Climate Change   Chapter 1: Introduction to
Climate Change   Climate change refers to significant, long -term
changes in the global climate. The term  "global climate" encompasses
the planet's overall weather patterns, including temperature,
precipitation, and wind patterns, over an extended period. Over the
past cent ury, human  ...


### Create vector store and retriever

In [6]:
# Create FaisVectorStore to store embeddings
fais_index = faiss.IndexFlatL2(EMBED_DIMENSION)
vector_store = FaissVectorStore(faiss_index=fais_index)

## Ingestion Pipelines

### Ingestion Pipeline with Sentence Splitter

In [7]:
base_pipeline = IngestionPipeline(
    transformations=[SentenceSplitter()],
    vector_store=vector_store
)

base_nodes = base_pipeline.run(documents=documents)

### Ingestion Pipeline with Sentence Window

In [8]:
node_parser = SentenceWindowNodeParser(
    # How many sentences on both sides to capture. 
    # Setting this to 3 results in 7 sentences.
    window_size=3,
    # the metadata key for to be used in MetadataReplacementPostProcessor
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence"
)

# Create a pipeline with defined document transformations and vectorstore
pipeline = IngestionPipeline(
    transformations=[node_parser],
    vector_store=vector_store,
)

windowed_nodes = pipeline.run(documents=documents)

## Querying

In [9]:
query = "Explain the role of deforestation and fossil fuels in climate change"

### Querying *without* Metadata Replacement 

In [10]:
# Create vector index from base nodes
base_index = VectorStoreIndex(base_nodes)

# Instantiate query engine from vector index
base_query_engine = base_index.as_query_engine(
    similarity_top_k=1,
)

# Send query to the engine to get related node(s)
base_response = base_query_engine.query(query)

print(base_response)

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/rag/lib/python3.10/site-packages/llama_index/llms/langchain/base.py:95: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  lc_message = self._llm.predict_messages(messages=lc_messages, **kwargs)


Deforestation and the burning of fossil fuels are two significant contributors to climate change. When forests are cleared, the stored carbon is released into the atmosphere, exacerbating the greenhouse effect. This process not only contributes to climate change but also leads to habitat loss and species extinction.

On the other hand, fossil fuels, particularly coal and oil, are major sources of CO2 emissions when burned for energy. The combustion of these fuels releases significant amounts of greenhouse gases, contributing to climate change and air quality issues. While natural gas is considered a cleaner-burning fossil fuel, its extraction and use still contribute to greenhouse gas emissions.

The combined impact of deforestation and fossil fuel use has severe consequences for the environment, including rising global temperatures, more frequent natural disasters, and altered ecosystems. It is essential to address these issues through sustainable practices, such as reforestation, ren

#### Print Metadata of the Retrieved Node

In [11]:
pprint(base_response.source_nodes[0].node.metadata)

{'creation_date': '2024-09-15',
 'file_name': 'Understanding_Climate_Change.pdf',
 'file_path': '/Users/michieldekoninck/code/Michiel-DK/rag_techniques/notebooks/../data/Understanding_Climate_Change.pdf',
 'file_size': 206372,
 'file_type': 'application/pdf',
 'last_modified_date': '2024-09-15',
 'page_label': '2'}


### Querying with Metadata Replacement
"Metadata replacement" intutively might sound a little off topic since we're working on the base sentences. But LlamaIndex stores these "before/after sentences" in the metadata data of the nodes. Therefore to build back up these windows of sentences we need Metadata replacement post processor.

In [12]:
# Create window index from nodes created from SentenceWindowNodeParser
windowed_index = VectorStoreIndex(windowed_nodes)

# Instantiate query enine with MetadataReplacementPostProcessor
windowed_query_engine = windowed_index.as_query_engine(
    similarity_top_k=1,
    node_postprocessors=[
        MetadataReplacementPostProcessor(
            target_metadata_key="window" # `window_metadata_key` key defined in SentenceWindowNodeParser
            )
        ],
)

# Send query to the engine to get related node(s)
windowed_response = windowed_query_engine.query(query)

print(windowed_response)

Deforestation and the burning of fossil fuels are two significant contributors to climate change. When trees are cut down or burned, the stored carbon is released into the atmosphere, exacerbating the greenhouse effect. This process reduces the number of trees that can absorb CO2, further contributing to the problem. On the other hand, fossil fuels, such as coal, oil, and gas, release massive amounts of carbon dioxide when burned, also contributing to the greenhouse effect and climate change.


#### Print Metadata of the Retrieved Node

In [13]:
# Window and original sentence are added to the metadata
pprint(windowed_response.source_nodes[0].node.metadata)

{'creation_date': '2024-09-15',
 'file_name': 'Understanding_Climate_Change.pdf',
 'file_path': '/Users/michieldekoninck/code/Michiel-DK/rag_techniques/notebooks/../data/Understanding_Climate_Change.pdf',
 'file_size': 206372,
 'file_type': 'application/pdf',
 'last_modified_date': '2024-09-15',
 'original_sentence': 'Deforestation in the \n'
                      'Amazon, Congo Basin, and Southeast Asia has significant '
                      'impacts on global carbon cycles \n'
                      'and biodiversity. ',
 'page_label': '2',
 'window': 'When trees are cut down \n'
           'for timber or to clear land for agriculture, this stored carbon is '
           'released back into the \n'
           'atmosphere.  Deforestation reduces the number of trees that can '
           'absorb CO2, exa cerbating the \n'
           'greenhouse effect.  \n'
           ' Tropical Deforestation  \n'
           'Tropical rainforests are particularly important for carbon '
           'stora